In [ ]:
# APP imports

from app import schemas, models
from app.core import db, conf 

# Standard library imports
from pathlib import Path
from pprint import pprint
import os
import json

# Third party imports
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# ENSURE THAT THE OPENAI_API_KEY IS SET and Define the model
os.environ['OPENAI_API_KEY'] = conf.openai.API_KEY
llm = ChatOpenAI(model='gpt-4')

### Load Seed data

Data is currently being loaded from the seeds directory.
- **TODO**: Load data from database

In [ ]:
seeds_dir = lead_docs_dir = Path(conf.settings.PUBLIC_ASSETS_DIR) / 'seeds'

with open(seeds_dir / 'leads' / 'leads.json', 'r') as lead_docs_json:
    lead_docs = json.load(lead_docs_json)

with open(seeds_dir / 'user.json') as users_json:
    user = json.load(users_json)

### Enrich Seed data
leads_schema

### Cover Letter & Resume Generation

Given a job lead and a user context, the system will generate a tailored cover letter and resume for the job based on the user's profile.

```python
def generate_cover_letter(user, lead):
    ...

def generate_resume(user, lead):
    ...
```

We will begin with the cover letter generation.

In [ ]:
str_output_parser = StrOutputParser()

cover_letter_generation_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI assistant that helps users generate cover letters for job applications."),
        ("user", "Hello, tailor my resume based on my background: {profile}"),
        ("system", """Great, please provide me with a job desctiption and I will generate a cover letter."""),
        ("user", "Awesome! Here are the job details:\n{job}"),
    ]
)

chain = cover_letter_generation_template | llm | str_output_parser

In [ ]:
chain.invoke({"profile": user, "job" : lead_docs[0]})

In [ ]:
def generate_cover_letter(profile, job, template) -> str:
    generation_template = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are a helpful AI assistant that helps users generate cover letters for job applications.",
            ),
            ("user", "Hello, tailor my resume based on my background: {profile}"),
            (
                "system",
                """Great, please provide me with a job desctiption and I will generate a cover letter from this template: {template}.""",
            ),
            ("user", "Awesome! Here are the job details:\n{job}"),
        ]
    )
    chain = generation_template | llm | str_output_parser
    return chain.invoke({"profile": profile, "job": job, "template": template})


def generate_resume(profile, job, template) -> str:
    generation_template = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are a helpful AI assistant that helps users generate resumes for job applications.",
            ),
            (
                "user",
                "Hello, tailor my resume based on my background: {profile}"),
            (
                "system",
                "Great, please provide me with a job desctiption and I will generate a resume from this template: {template}.",
            ),
            (
                "user",
                "Awesome! Here are the job details:\n{job}"
            ),
        ]
    )
    chain = generation_template | llm | str_output_parser
    return chain.invoke({"profile": profile, "job": job, "template": template})



Next, we integrate the generation function parameters and return types to use `app.schemas`  pydantic types.

In [ ]:
def generate_cover_letter(profile:schemas.UserProfileRead, lead:schemas.LeadRead, template: schemas.CoverLetterRead) -> str:
    ...


def generate_resume(profile:schemas.UserProfileRead, lead:schemas.LeadRead, template: schemas.ResumeRead) -> str:
    ...

print(schemas.UserProfileRead.model_json_schema())
print(schemas.LeadRead.model_json_schema())

### Lead Enrichment Chain

The lead enrichment chain is a series of functions that enrich the lead data with additional information. The chain is defined as a list of functions that take a lead and return an enriched lead.

The routes `/leads/enrich` and `/leads/enrich/{lead_id}` are for enriching a single lead and all leads, respectively.

Enrichment off all leads is executed as a background task.



In [ ]:
def enrich_lead(lead: schemas.LeadRead) -> schemas.LeadUpdate:
    ...